# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [12]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

from week1.solutions.day1_with_ollama import messages

In [19]:
# Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')
#
# if not api_key:
#     print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
# elif not api_key.startswith("sk-proj-") and api_key != 'ollama':
#     print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
# else:
#     print("API key found and looks good so far!")

# I'm using Ollama for this notebook and running a local Llama 3.2 model
OLLAMA_BASE_URL = "http://localhost:11434/v1"
MODEL = 'llama3.2:latest'
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [7]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [14]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [15]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [16]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [20]:
def select_relevant_links(url):
    messages = [
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": get_links_user_prompt(url)}
    ]

    response = ollama.chat.completions.create(
        model=MODEL,
        messages=messages,
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [21]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'About page (personal)',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'Company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'}]}

In [22]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = ollama.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [23]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling llama3.2:latest
Found 6 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'About page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'Companies/nebula.io/',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'Careers/Jobs', 'url': None},
  {'type': 'Blog/Posts', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn Profile',
   'url': 'https://www.linkedin.com/in/eddonner/',
   "#{redir:edwarddonnerlinkedin} #<#redir link> - {redir url}  ^{redir text} ^{redir attrtext} ^redir > ^\x1b{redir attr:facebook} \x1bfound in: ^\n ^\x1b{redir attr:twitter} \x1bfound in: ^\n ^\x1b{redir attr:text} \x1bfound in: ^\r?\n\r· #<#redir link>- {\redir:facebook text}\n\n^{\newurl 'edwarddonnerlinkedinprofile'> ^\x1b# {redirattr:linkedin attrtext} \x1bfound in: ^}\x1b": {}}]}

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [24]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [25]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling llama3.2:latest
Found 4 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-Image
Updated
about 19 hours ago
•
2.44k
•
704
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
8 days ago
•
44.1k
•
686
Lightricks/LTX-2
Updated
1 day ago
•
1.19M
•
1.05k
openbmb/AgentCPM-Explore
Updated
2 days ago
•
315
•
293
Kijai/LTXV2_comfy
Updated
1 day ago
•
41.1k
•
283
Browse 2M+ models
Spaces
Running
on
Zero
Featured
798
Qwen Image Multiple Angles 3D Camera
🎥
798
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
MCP
1.48k
Z Image Turbo
🖼
1.48k
Generate stunning images from text 

In [26]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [28]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [29]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2:latest
Found 10 relevant links


ConnectionError: HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("HTTPSConnection(host='blog.huggingface.co', port=443): Failed to resolve 'blog.huggingface.co' ([Errno 8] nodename nor servname provided, or not known)"))

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face: Building the Future of AI Together

---

## Who We Are

Hugging Face is the leading AI community and collaboration platform dedicated to advancing machine learning. We provide a dynamic hub where researchers, engineers, and enthusiasts come together to create, share, and deploy cutting-edge machine learning models, datasets, and applications. Our mission is to empower the next generation of machine learning professionals to build an open, collaborative, and ethical AI future.

---

## What We Offer

- **2 Million+ Models:** Explore, use, and contribute to one of the world’s largest collections of open-source machine learning models across all data modalities including text, image, video, audio, and even 3D.

- **500,000+ Datasets:** Access an enormous variety of public datasets that fuel advanced AI model training and research.

- **1 Million+ Applications:** Discover AI-powered applications built by the community and personalize your exploration of AI innovations.

- **Spaces:** Host and interact with machine learning applications in a zero-setup environment, allowing rapid experimentation and sharing.

- **HF Open Source Stack:** Leverage a robust and fast open-source toolkit to accelerate your ML development cycle.

---

## Our Community

Hugging Face thrives on its vibrant global community of machine learning practitioners, researchers, and developers. We foster a collaborative culture that values:

- **Open Innovation:** Everyone can share their models, datasets, and applications for free, contributing to collective progress.

- **Ethical AI:** Commitment to building AI tools that are transparent, fair, and responsibly developed.

- **Learning & Growth:** Providing resources and a platform to build your personal ML portfolio and reputation.

- **Diversity & Inclusion:** Encouraging participation from diverse backgrounds to enrich the AI ecosystem.

---

## For Customers & Enterprise

We support organizations in scaling their AI initiatives with enterprise-grade solutions and dedicated support while maintaining the spirit of openness and collaboration. Whether you’re building next-gen products or conducting research, Hugging Face offers the tools to accelerate your machine learning roadmap.

---

## Careers at Hugging Face

Join a passionate team shaping the future of machine learning! We offer opportunities for engineers, researchers, product managers, and community builders eager to work in a fast-paced environment that values creativity, impact, and open collaboration. Be part of an innovative culture dedicated to ethical AI and global community building.

---

## Get Involved

- Explore AI models and apps at [huggingface.co](https://huggingface.co)
- Sign up to contribute and build your ML profile.
- Engage with our growing community through forums and collaborative projects.
- Access full documentation and developer tools.

---

## Brand & Identity

Hugging Face represents openness and creativity in AI:

- Primary colors: Yellow (#FFD21E, #FF9D00) and Gray (#6B7280)
- Branding assets including logos are available to promote our values and identity.

---

*Hugging Face: The Home of Machine Learning Collaboration*  
Together, we move AI forward. Join us!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the leading AI community and platform dedicated to building the future of machine learning. It empowers a global community of developers, researchers, and enterprises to discover, create, share, and collaborate on machine learning models, datasets, and applications.  

With over **2 million models**, and hundreds of thousands of datasets and applications, Hugging Face covers a wide spectrum of AI modalities including text, image, video, audio, and even 3D. It is not just a platform but a vibrant community where machine learning practitioners grow their skills, share projects, and build professional portfolios.

---

## What Hugging Face Offers

- **Models:** Access and contribute to over 2 million pre-trained AI models.
- **Datasets:** Browse and host more than 500,000 datasets.
- **Spaces:** Deploy interactive machine learning applications with ease.
- **Community:** Engage with AI enthusiasts, experts, and collaborators worldwide.
- **Open Source Stack:** Utilize powerful open-source tools to accelerate ML development.
- **Multi-Modal Support:** Work with diverse data types—text, image, video, audio, and 3D.

---

## For Enterprises & Teams

Hugging Face offers tailored solutions for organizations looking to scale AI initiatives securely and efficiently:

- **Team & Enterprise Plans:** Starting at $20 per user/month, or customized enterprise contracts.
- **Enterprise-Grade Security:** Single Sign-On (SSO), advanced security policies, detailed audit logs.
- **Private Storage & Data Management:** 1 TB private storage per user plus options for additional storage.
- **Granular Access Control:** Resource groups and centralized token management.
- **Advanced Compute:** Access to ZeroGPU and options to boost quotas by 5×.
- **Billing & Analytics:** Manage budgets, monitor usage, and get detailed analytics through a unified dashboard.
- **Priority Support:** Enterprise customers receive dedicated priority support.

---

## Pricing Highlights

- **Free Tier:** Community access to models, datasets, spaces, and collaboration tools.
- **Pro Plan ($9/month):** Enhanced features for individuals including:
  - 10× private storage capacity
  - 20× inference credits
  - 8× ZeroGPU quota and higher queue priority
  - Spaces Dev Mode & ZeroGPU hosting
  - Ability to publish blog articles on HF profile
- **Team Plan ($20/user/month):** Onboarding for teams with features like SSO and enterprise integrations.

---

## Company Culture

Hugging Face fosters an open, collaborative, and inclusive culture where innovation and sharing thrive. Being community-driven, it enables contributors worldwide to build knowledge, share breakthroughs, and accelerate AI researches and applications together.

- **Community-Centric:** The core of Hugging Face is its vibrant, global AI community.
- **Open Source Enthusiasts:** Heavy investment in open source AI tools and resources.
- **Innovation & Accessibility:** Committed to making cutting-edge AI technology accessible to all.
- **Developer Friendly:** Building tools and platforms that empower machine learning practitioners at every level.

---

## Careers at Hugging Face

Hugging Face is continuously growing and welcomes talented individuals passionate about advancing AI technology. Career opportunities include roles in:

- Machine Learning Research and Engineering
- Software Development and Platform Engineering
- Customer Success and Enterprise Solutions
- Community Management and Developer Relations

Working at Hugging Face means contributing to a future where AI is more collaborative, transparent, and impactful globally.

---

## Join Hugging Face

Whether you are an AI researcher, developer, enterprise leader, or enthusiast, Hugging Face invites you to:

- **Explore and build** with state-of-the-art models and datasets
- **Deploy** AI applications easily on the community-driven Spaces platform
- **Collaborate** with leading scientists and engineers
- **Scale** AI solutions securely in your organization with enterprise features

**Sign up today** and become part of the AI community building the future.

---

### Website: [huggingface.co](https://huggingface.co)  
### Contact: enterprise@huggingface.co (for sales and enterprise inquiries)  
### Pricing & Plans: [huggingface.co/pricing](https://huggingface.co/pricing)

---

*Hugging Face – The AI community building the future.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>